In [3]:
import lithops
import xml.etree.ElementTree as ET
import pandas as pd
import bz2
import boto3

In [ ]:
from lithops import Storage
from lithops import FunctionExecutor
from lithops.multiprocessing import Pool

In [ ]:
BUCKET_NAME = "practica-sd"
storage = lithops.Storage()
wiki_list = storage.list_objects("practica-sd")
iterdata = []

In [ ]:
for i in wiki_list:
    key=i.get("Key")
    if key.startswith("Wikipedia") and key.endswith(".xml"):
        iterdata.append(key)

In [4]:
cols = ["id", "title", "text"]

In [ ]:
def read_wiki(wiki):
    xml_code = storage.get_object(BUCKET_NAME, wiki)
    root = ET.fromstring(xml_code)
    rows = []
    for i in root:
        if i.tag == "page":
            ns = i.find("ns").text
            if ns == "0":
                id=i.find("id").text
                title=i.find("title").text
                text=i.find("revision").find("text").text
                rows.append({"id" : id, "title" : title, "text" : text})
    return rows

In [ ]:
def my_reduce_function(results):
    rows = []
    for map_result in results:
        rows = rows + map_result
    return rows

In [ ]:
if __name__ == "__main__":
    fexec = lithops.FunctionExecutor()
    fexec.map_reduce(read_wiki, iterdata, my_reduce_function, reducer_wait_local=True)
    df = pd.DataFrame(fexec.get_result(), columns=cols)
    storage.put_object("practica-sd", "WikipediaES", df.to_csv())

In [ ]:
configuracion = {'lithops' : {'storage' : 'ibm_cos'},
          'ibm_cos': {'region': 'REGION', 'api_key': 'API_KEY'},
          'redis': {'host': 'HOST', 'port':'PORT'}}

In [ ]:
storage = Storage(config=configuracion)
storage = Storage(backend='redis') #esto creará una instancia de almacenamiento de redis

In [ ]:
dataS = storage.get_object("practica-sd", "WikipediaES")

In [ ]:
obj_metadata = storage.head_object("practica-sd", "WikipediaES")

In [ ]:
storage.delete_object("practica-sd", "WikipediaES")

In [ ]:
storage.head_bucket("practica-sd")

In [ ]:
storage.list_objects("practica-sd", prefix='temp/')

In [ ]:
storage.list_keys("practica-sd")

In [ ]:
boto3_client = storage.get_client()

In [ ]:
cobj = storage.put_cloudobject(df.to_csv(),"practica-sd","WikipediaES")
dataS = storage.get_cloudobject(cobj)

In [ ]:
storage.delete_cloudobject(cobj)

In [ ]:
if __name__ == '__main__':
    p = Process(target=function,args=(name,),kwargs={'language':'english'})
    p.start()

In [ ]:
if __name__ == '__main__':
    pool = Pool(processes= 4)
    count = pool.map(is_inside,part_count)

In [ ]:
def get_file(key,storage):
    return storage.get_object(bucket=BUCKET_NAME, key="WikipediaES")

In [ ]:
if __name__ == "__main__":
    storage = Storage()
    storage.put_object(bucket=BUCKET_NAME,
                      key="WikipediaES",
                      body=df.to_csv())

In [ ]:
with FunctionExecutor() as fexec:
    fut = fexec.call_async(get_file, "WikipediaES")
    print(fut.result())